In [12]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer, ViTImageProcessor
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

In [23]:
# data_path = os.getcwd() + "/coco2017_data/"
# ds = datasets.load_dataset(path = "make_coco_train_data", name = "2017", data_dir=data_path)

data_path = "D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/"
ds = datasets.load_dataset(path = "make_coco_train_data", name = "2017", data_dir=data_path)

"""
DatasetDict({
    train: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 80
    })
    test: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 16
    })
})
"""
ds

Dataset make_coco_train_data downloaded and prepared to C:/Users/DRCL/.cache/huggingface/datasets/make_coco_train_data/2017-6486e79d16f782ce/0.0.0/102355ff62a645404d15fc83bbaa46137b14eeb4c8396d78238d39d0a8775fe4. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 186.63it/s]


DatasetDict({
    train: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 80
    })
    test: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 16
    })
})

In [24]:
new_data = [{"image_id" : -1, 
            "caption_id" : -1, 
            "height" : 720, 
            "width" : 1280, 
            "file_name" : "test.png", 
            "coco_url" : "none", 
            "image_path" : os.getcwd() + "/images/test.png",
            "caption": "The man is building a fire."},
            {"image_id" : -1, 
            "caption_id" : -1, 
            "height" : 720, 
            "width" : 1280, 
            "file_name" : "fire_00007690.jpg", 
            "coco_url" : "none", 
            "image_path" : os.getcwd() + "/images/fire_00007690.jpg",
            "caption": "The pot is on fire."},
            {"image_id" : -1, 
            "caption_id" : -1, 
            "height" : 720, 
            "width" : 1280, 
            "file_name" : "fire_00007622.jpg", 
            "coco_url" : "none", 
            "image_path" : os.getcwd() + "/images/fire_00007622.jpg",
            "caption": "A person is sitting next to a pot."},
            {"image_id" : -1, 
            "caption_id" : -1, 
            "height" : 720, 
            "width" : 1280, 
            "file_name" : "fire_00007628.jpg", 
            "coco_url" : "none", 
            "image_path" : os.getcwd() + "/images/fire_00007628.jpg",
            "caption": "A person is standing next to a pot and the pot is on fire."}]
new_ds = ds

for data in new_data:
    new_ds["train"] = new_ds["train"].add_item(data)

new_ds

DatasetDict({
    train: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 84
    })
    validation: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 80
    })
    test: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 16
    })
})

In [25]:
for i in range(84):
    print(new_ds['train'][i]['image_path'])

D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000074.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000074.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000074.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000074.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000074.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000073.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000073.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000073.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000073.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000073.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000042.jpg
D:\yoseph\DataSet\Captioning_dataset/coco2017_data_test/train2017\000000000042.jpg
D:\y

In [13]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# GPT2 only has bos/eos tokens but not decoder_start/pad tokens
tokenizer.pad_token = tokenizer.eos_token

# update the model config
model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [15]:
output_dir = "./model/test-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model/test-model\\tokenizer_config.json',
 './model/test-model\\special_tokens_map.json',
 './model/test-model\\vocab.json',
 './model/test-model\\merges.txt',
 './model/test-model\\added_tokens.json',
 './model/test-model\\tokenizer.json')

In [14]:
# from PIL import Image
import cv2
import numpy as np

# text preprocessing step
def tokenization_fn(captions, max_target_length):
    """Run tokenization on captions."""
    labels = tokenizer(captions, padding="max_length", max_length=max_target_length).input_ids

    return labels

def feature_extraction_fn(image_paths, check_image=True):
    """
    Run feature extraction on images
    If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
    Otherwise, an exception will be thrown.
    """

    model_inputs = {}

    if check_image:
        images = []
        to_keep = []
        for image_file in image_paths:
            try:
                # img = Image.open(image_file)
                img = cv2.imread(image_file)
                if len(img.shape) == 2:  # 이미지가 2차원인 경우
                    img = np.expand_dims(img, axis=2)  # 차원을 확장하여 3차원으로 만듦

                images.append(img)
                to_keep.append(True)
            except Exception:
                to_keep.append(False)
    else:
        images = [Image.open(image_file) for image_file in image_paths]

    encoder_inputs = feature_extractor(images=images, return_tensors="np")

    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    """Run tokenization + image feature extraction"""
    image_paths = examples['image_path']
    captions = examples['caption']    
    
    model_inputs = {}
    # This contains image path column
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths, check_image=check_image)

    return model_inputs

In [26]:
# processed_dataset = ds.map(function=preprocess_fn, batched=True, fn_kwargs={"max_target_length": 128}, remove_columns=ds['train'].column_names)
processed_dataset = new_ds.map(function=preprocess_fn, batched=True, fn_kwargs={"max_target_length": 2000}, remove_columns=ds['train'].column_names)

# processed_dataset

In [7]:
processed_dataset.save_to_disk('./processed_dataset')

test = processed_dataset.load_from_disk('./processed_dataset')

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(predict_with_generate=True, evaluation_strategy="epoch", per_device_train_batch_size=4, per_device_eval_batch_size=4, output_dir="./model/image-captioning-output",num_train_epochs = 1)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
import evaluate
metric = evaluate.load("rouge")

import numpy as np

ignore_pad_token_for_loss = True

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    
    return result

In [10]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(model=model, tokenizer=feature_extractor, args=training_args, compute_metrics=compute_metrics, train_dataset=processed_dataset['train'], eval_dataset=processed_dataset['validation'], data_collator=default_data_collator,)

trainer.train()

c:\Users\DRCL\anaconda3\envs\caption\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/21 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [11]:
trainer.save_model("./image-captioning-output")
tokenizer.save_pretrained("./image-captioning-output")

('./image-captioning-output\\tokenizer_config.json',
 './image-captioning-output\\special_tokens_map.json',
 './image-captioning-output\\vocab.json',
 './image-captioning-output\\merges.txt',
 './image-captioning-output\\added_tokens.json',
 './image-captioning-output\\tokenizer.json')

In [22]:

from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("./image-captioning-output")
feature_extractor = ViTImageProcessor.from_pretrained("./image-captioning-output")
tokenizer = AutoTokenizer.from_pretrained("./image-captioning-output")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 32
num_beams = 12
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds




In [23]:
predict_step(['./images/Sunset_TB_360_00-00-03.jpg']) 

['A giraffe standing next to a tree.']